# Miscellaneous Utilities
The follow demonstrates some miscellaneous utilities included in ObsPlus.

## Geodetics
It is often necessary to calculate geometric parameters (distance, azimuth, etc.) for pairs of entities in two different groups. For example, distance from each event in a catalog to each receiver in an inventory. ObsPlus provides a simple class for completing this task.

In [ ]:
# Load the catalog and inventory from the crandall dataset
import obsplus
from obsplus.utils.geodetics import SpatialCalculator

crandall = obsplus.load_dataset("crandall_test")
cat = crandall.event_client.get_events()
inv = crandall.station_client.get_stations()

In [ ]:
# init a SpatialCalculator instance (defaults to Earth's params)
spatial_calc = SpatialCalculator()

In [ ]:
# create distance dataframe
df = spatial_calc(entity_1=cat, entity_2=inv)
df.head()

Since a multi-index is used it provides a fairly intuitive way to look up particular event-channel pairs using a tuple of (event_id, seed_id) in conjunction with the `.loc` DataFrame attribute like so:

In [ ]:
event_id = str(cat[0].resource_id)
seed_id = "UU.MPU..HHZ"


In [ ]:
# or just get a particular parameter

The distances can be converted to km and the distrtibutions can be described.

In [ ]:
# Convert add km columns and delete m columns
m_columns = [x for x in df.columns if x.endswith("_m")]
km_columns = [x.replace("_m", "_km") for x in m_columns]

df_km = df.assign(**{x: df[y] / 1000.0 for x, y in zip(km_columns, m_columns)}).drop(
    columns=m_columns
)

# Calculate stats for source reseiver distances
df_km.describe().round(decimals=2)

## Time

Working with dates/times can be frustrating, especially since (as of 2020) ObsPy, numpy, and pandas all use slightly different methods for working with time. ObsPlus provides some utilities to make things a little easier.

In [ ]:

import numpy as np
import obspy

import obsplus
from obsplus.utils import to_datetime64, to_timedelta64, to_utc

**Note:**  All ObsPlus datafames use numpy/pandas datatypes. 

In [ ]:
df = obsplus.events_to_df(obspy.read_events())

In [ ]:
df["time"]

### Numpy and ObsPy time differences

One difference between numpy's `datatime64` and ObsPy's `UTCDateTime` is how offsets are applied. For ObsPy, numbers are simply taken as seconds, but numpy requires explicitly using `timedeta64` instances.

In [ ]:
time_str = "2020-01-03T11:00:00"
utc = obspy.UTCDateTime(time_str)
dt64 = np.datetime64(time_str, "ns")

In [ ]:
# add 1/2 second to UTCDateTime
utc2 = utc + 0.5

In [ ]:
# however doing the same thing with datetime64 raises a TypeError
try:
    dt64 + 0.5
except TypeError:
    pass

In [ ]:
# so you need to use a timedelta64
dt64_2 = dt64 + np.timedelta64(500, "ms")

In [ ]:
# This is, of course, also the case with datetime64 columns
df["time"] + np.timedelta64(500, "ms")

ObsPlus provides some common converters between ObsPy and Pandas/Numpy time objects.

### Converting between ObsPy and Numpy time datatypes
Conversion between ObsPy `UTCDateTime` and numpy `datetime64` objects when needed.

In [ ]:
# Convert a time column to an array of ObsPy UTCDateTime objects
utc_array = to_utc(df["time"])

In [ ]:
# Convert back to datetime64
dt64_array = to_datetime64(utc_array)

<div class="alert alert-warning">

**Note**: `datetime64` arrays are much more efficient in terms of memory usage and computational efficiency than arrays of `UTCDateTime` *objects*.

</div>

If you prefer not to manually define `timedelta64` to perform offsets, `to_timedelta64` simply converts a real number to an offset in seconds.

In [ ]:
to_timedelta64(3.255)